In [33]:
"""
Setup the strides, padding and filter weight/bias such that
the output shape is (1, 2, 2, 3).
"""
import tensorflow as tf
import numpy as np

# `tf.nn.conv2d` requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape((1, 4, 4, 1))

X = tf.constant(x)

def conv2d(input_data):
    # Filter (weights and bias)
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    # The shape of the filter bias is (output_depth,)
    # TODO: Define the filter weights `F_W` and filter bias `F_b`.
    # NOTE: Remember to wrap them in `tf.Variable`, they are trainable parameters after all.
    x = tf.placeholder(tf.float32, input_data)
    
    F_W = tf.Variable(tf.truncated_normal(2,2,1,3))
    F_b = tf.Variable(tf.zeros(3))
    # TODO: Set the stride for each dimension (batch_size, height, width, depth)
    strides = [1, 2, 2, 1]
    # TODO: set the padding, either 'VALID' or 'SAME'.
    padding = 'VALID'
    # https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#conv2d
    # `tf.nn.conv2d` does not include the bias computation so we have to add it ourselves after.
    conv_layer = tf.nn.conv2d(x, F_W, strides, padding)
    conv_layer = tf.nn.bias_add(conv_layer, F_b)
    conv_layer = tf.nn.relu(conv_layer)
    conv_layer = tf.nn.max_pool(conv_layer,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')
    return output 

out = conv2d(X)











TypeError: Error converting shape to a TensorShape: `Tensor` objects are not iterable when eager execution is not enabled. To iterate over this tensor use `tf.map_fn`..

In [32]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch  1, Batch   1 - Loss: 56607.9844 Validation Accuracy: 0.117188
Epoch  1, Batch   2 - Loss: 44390.4023 Validation Accuracy: 0.101562
Epoch  1, Batch   3 - Loss: 42236.6406 Validation Accuracy: 0.082031
Epoch  1, Batch   4 - Loss: 38468.4062 Validation Accuracy: 0.089844
Epoch  1, Batch   5 - Loss: 30575.8867 Validation Accuracy: 0.074219
Epoch  1, Batch   6 - Loss: 36566.7461 Validation A

Epoch  1, Batch 109 - Loss:  5590.1812 Validation Accuracy: 0.578125
Epoch  1, Batch 110 - Loss:  4501.5332 Validation Accuracy: 0.562500
Epoch  1, Batch 111 - Loss:  4790.9595 Validation Accuracy: 0.558594
Epoch  1, Batch 112 - Loss:  4397.1035 Validation Accuracy: 0.566406
Epoch  1, Batch 113 - Loss:  4714.3662 Validation Accuracy: 0.558594
Epoch  1, Batch 114 - Loss:  4171.8906 Validation Accuracy: 0.570312
Epoch  1, Batch 115 - Loss:  4026.8779 Validation Accuracy: 0.570312
Epoch  1, Batch 116 - Loss:  4659.1787 Validation Accuracy: 0.566406
Epoch  1, Batch 117 - Loss:  4598.9502 Validation Accuracy: 0.562500
Epoch  1, Batch 118 - Loss:  3593.8845 Validation Accuracy: 0.562500
Epoch  1, Batch 119 - Loss:  3190.9126 Validation Accuracy: 0.554688
Epoch  1, Batch 120 - Loss:  4332.8853 Validation Accuracy: 0.570312
Epoch  1, Batch 121 - Loss:  3673.2612 Validation Accuracy: 0.578125
Epoch  1, Batch 122 - Loss:  2779.9319 Validation Accuracy: 0.570312
Epoch  1, Batch 123 - Loss:  3874.

Epoch  1, Batch 228 - Loss:  2930.9500 Validation Accuracy: 0.652344
Epoch  1, Batch 229 - Loss:  2819.3213 Validation Accuracy: 0.644531
Epoch  1, Batch 230 - Loss:  1587.2391 Validation Accuracy: 0.636719
Epoch  1, Batch 231 - Loss:  1603.3633 Validation Accuracy: 0.648438
Epoch  1, Batch 232 - Loss:  2673.3894 Validation Accuracy: 0.656250
Epoch  1, Batch 233 - Loss:  2548.4553 Validation Accuracy: 0.656250
Epoch  1, Batch 234 - Loss:  2280.1233 Validation Accuracy: 0.648438
Epoch  1, Batch 235 - Loss:  2099.3765 Validation Accuracy: 0.648438
Epoch  1, Batch 236 - Loss:  2412.0029 Validation Accuracy: 0.644531
Epoch  1, Batch 237 - Loss:  1923.7202 Validation Accuracy: 0.648438
Epoch  1, Batch 238 - Loss:  2448.0249 Validation Accuracy: 0.652344
Epoch  1, Batch 239 - Loss:  2041.2622 Validation Accuracy: 0.644531
Epoch  1, Batch 240 - Loss:  2464.2534 Validation Accuracy: 0.636719
Epoch  1, Batch 241 - Loss:  2225.4490 Validation Accuracy: 0.640625
Epoch  1, Batch 242 - Loss:  1884.

Epoch  1, Batch 347 - Loss:  1636.0112 Validation Accuracy: 0.660156
Epoch  1, Batch 348 - Loss:  1312.1320 Validation Accuracy: 0.664062
Epoch  1, Batch 349 - Loss:  1503.1121 Validation Accuracy: 0.664062
Epoch  1, Batch 350 - Loss:  1875.5004 Validation Accuracy: 0.671875
Epoch  1, Batch 351 - Loss:  1756.1155 Validation Accuracy: 0.664062
Epoch  1, Batch 352 - Loss:  1288.7676 Validation Accuracy: 0.664062
Epoch  1, Batch 353 - Loss:  1696.1758 Validation Accuracy: 0.667969
Epoch  1, Batch 354 - Loss:  1052.0350 Validation Accuracy: 0.675781
Epoch  1, Batch 355 - Loss:  1348.9929 Validation Accuracy: 0.671875
Epoch  1, Batch 356 - Loss:  2483.0542 Validation Accuracy: 0.667969
Epoch  1, Batch 357 - Loss:  1741.0319 Validation Accuracy: 0.667969
Epoch  1, Batch 358 - Loss:  2300.0146 Validation Accuracy: 0.664062
Epoch  1, Batch 359 - Loss:  1402.6901 Validation Accuracy: 0.664062
Epoch  1, Batch 360 - Loss:   808.4733 Validation Accuracy: 0.675781
Epoch  1, Batch 361 - Loss:  1715.

Epoch  2, Batch  37 - Loss:  1152.9495 Validation Accuracy: 0.703125
Epoch  2, Batch  38 - Loss:  1112.7035 Validation Accuracy: 0.703125
Epoch  2, Batch  39 - Loss:  1649.6332 Validation Accuracy: 0.710938
Epoch  2, Batch  40 - Loss:  1353.7784 Validation Accuracy: 0.710938
Epoch  2, Batch  41 - Loss:  1077.3690 Validation Accuracy: 0.710938
Epoch  2, Batch  42 - Loss:  1812.0173 Validation Accuracy: 0.710938
Epoch  2, Batch  43 - Loss:  1008.3127 Validation Accuracy: 0.703125
Epoch  2, Batch  44 - Loss:  1414.6952 Validation Accuracy: 0.707031
Epoch  2, Batch  45 - Loss:  1245.7134 Validation Accuracy: 0.699219
Epoch  2, Batch  46 - Loss:  1174.0986 Validation Accuracy: 0.703125
Epoch  2, Batch  47 - Loss:  1415.4698 Validation Accuracy: 0.707031
Epoch  2, Batch  48 - Loss:  1362.6715 Validation Accuracy: 0.714844
Epoch  2, Batch  49 - Loss:  1309.4734 Validation Accuracy: 0.722656
Epoch  2, Batch  50 - Loss:  1204.2595 Validation Accuracy: 0.722656
Epoch  2, Batch  51 - Loss:  1490.

...

Epoch  9, Batch 366 - Loss:   177.8166 Validation Accuracy: 0.816406
Epoch  9, Batch 367 - Loss:   130.4560 Validation Accuracy: 0.808594
Epoch  9, Batch 368 - Loss:   261.2777 Validation Accuracy: 0.812500
Epoch  9, Batch 369 - Loss:   222.0063 Validation Accuracy: 0.808594
Epoch  9, Batch 370 - Loss:   122.6659 Validation Accuracy: 0.812500
Epoch  9, Batch 371 - Loss:   110.2198 Validation Accuracy: 0.812500
Epoch  9, Batch 372 - Loss:   226.3024 Validation Accuracy: 0.804688
Epoch  9, Batch 373 - Loss:   160.2926 Validation Accuracy: 0.812500
Epoch  9, Batch 374 - Loss:   166.8565 Validation Accuracy: 0.808594
Epoch  9, Batch 375 - Loss:   183.1503 Validation Accuracy: 0.812500
Epoch  9, Batch 376 - Loss:   105.7465 Validation Accuracy: 0.812500
Epoch  9, Batch 377 - Loss:   194.1305 Validation Accuracy: 0.808594
Epoch  9, Batch 378 - Loss:   175.3045 Validation Accuracy: 0.808594
Epoch  9, Batch 379 - Loss:   226.3650 Validation Accuracy: 0.816406
Epoch  9, Batch 380 - Loss:   171.

Epoch 10, Batch  56 - Loss:   241.7295 Validation Accuracy: 0.804688
Epoch 10, Batch  57 - Loss:   236.0987 Validation Accuracy: 0.804688
Epoch 10, Batch  58 - Loss:   210.2122 Validation Accuracy: 0.812500
Epoch 10, Batch  59 - Loss:   150.8712 Validation Accuracy: 0.800781
Epoch 10, Batch  60 - Loss:   303.0605 Validation Accuracy: 0.812500
Epoch 10, Batch  61 - Loss:   268.1485 Validation Accuracy: 0.808594
Epoch 10, Batch  62 - Loss:   196.4465 Validation Accuracy: 0.804688
Epoch 10, Batch  63 - Loss:   221.3719 Validation Accuracy: 0.808594
Epoch 10, Batch  64 - Loss:   299.0035 Validation Accuracy: 0.808594
Epoch 10, Batch  65 - Loss:   281.2787 Validation Accuracy: 0.812500
Epoch 10, Batch  66 - Loss:   188.4172 Validation Accuracy: 0.804688
Epoch 10, Batch  67 - Loss:   199.7552 Validation Accuracy: 0.804688
Epoch 10, Batch  68 - Loss:   157.6029 Validation Accuracy: 0.808594
Epoch 10, Batch  69 - Loss:   299.6748 Validation Accuracy: 0.808594
Epoch 10, Batch  70 - Loss:   260.

Epoch 10, Batch 175 - Loss:   155.6966 Validation Accuracy: 0.800781
Epoch 10, Batch 176 - Loss:   329.1044 Validation Accuracy: 0.796875
Epoch 10, Batch 177 - Loss:   204.0319 Validation Accuracy: 0.796875
Epoch 10, Batch 178 - Loss:   275.2805 Validation Accuracy: 0.792969
Epoch 10, Batch 179 - Loss:   154.5867 Validation Accuracy: 0.796875
Epoch 10, Batch 180 - Loss:   136.8470 Validation Accuracy: 0.808594
Epoch 10, Batch 181 - Loss:   228.5178 Validation Accuracy: 0.804688
Epoch 10, Batch 182 - Loss:   364.5706 Validation Accuracy: 0.812500
Epoch 10, Batch 183 - Loss:   228.8723 Validation Accuracy: 0.812500
Epoch 10, Batch 184 - Loss:   199.7158 Validation Accuracy: 0.816406
Epoch 10, Batch 185 - Loss:   212.2490 Validation Accuracy: 0.812500
Epoch 10, Batch 186 - Loss:   202.7941 Validation Accuracy: 0.812500
Epoch 10, Batch 187 - Loss:   215.2479 Validation Accuracy: 0.812500
Epoch 10, Batch 188 - Loss:   198.3094 Validation Accuracy: 0.812500
Epoch 10, Batch 189 - Loss:   170.

Epoch 10, Batch 294 - Loss:   227.3966 Validation Accuracy: 0.800781
Epoch 10, Batch 295 - Loss:   176.6457 Validation Accuracy: 0.804688
Epoch 10, Batch 296 - Loss:   259.8664 Validation Accuracy: 0.808594
Epoch 10, Batch 297 - Loss:   195.2739 Validation Accuracy: 0.808594
Epoch 10, Batch 298 - Loss:   141.5664 Validation Accuracy: 0.808594
Epoch 10, Batch 299 - Loss:   272.7214 Validation Accuracy: 0.812500
Epoch 10, Batch 300 - Loss:   270.5336 Validation Accuracy: 0.804688
Epoch 10, Batch 301 - Loss:   288.1006 Validation Accuracy: 0.804688
Epoch 10, Batch 302 - Loss:   173.9643 Validation Accuracy: 0.800781
Epoch 10, Batch 303 - Loss:   174.8675 Validation Accuracy: 0.800781
Epoch 10, Batch 304 - Loss:   214.4495 Validation Accuracy: 0.812500
Epoch 10, Batch 305 - Loss:   150.0299 Validation Accuracy: 0.804688
Epoch 10, Batch 306 - Loss:   204.4243 Validation Accuracy: 0.804688
Epoch 10, Batch 307 - Loss:   182.6501 Validation Accuracy: 0.808594
Epoch 10, Batch 308 - Loss:   100.

Epoch 10, Batch 413 - Loss:   161.8804 Validation Accuracy: 0.800781
Epoch 10, Batch 414 - Loss:   225.0601 Validation Accuracy: 0.800781
Epoch 10, Batch 415 - Loss:   109.0868 Validation Accuracy: 0.796875
Epoch 10, Batch 416 - Loss:   172.5969 Validation Accuracy: 0.796875
Epoch 10, Batch 417 - Loss:   173.6858 Validation Accuracy: 0.800781
Epoch 10, Batch 418 - Loss:   182.3983 Validation Accuracy: 0.796875
Epoch 10, Batch 419 - Loss:   210.6260 Validation Accuracy: 0.796875
Epoch 10, Batch 420 - Loss:   198.3724 Validation Accuracy: 0.796875
Epoch 10, Batch 421 - Loss:   192.5678 Validation Accuracy: 0.796875
Epoch 10, Batch 422 - Loss:   244.3533 Validation Accuracy: 0.796875
Epoch 10, Batch 423 - Loss:   158.9733 Validation Accuracy: 0.796875
Epoch 10, Batch 424 - Loss:   196.6777 Validation Accuracy: 0.800781
Epoch 10, Batch 425 - Loss:   224.7423 Validation Accuracy: 0.796875
Epoch 10, Batch 426 - Loss:   241.1812 Validation Accuracy: 0.796875
Epoch 10, Batch 427 - Loss:   184.